## Chain using LangGraph

In [1]:
from dotenv import load_dotenv
import os
load_dotenv("./.env")

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint

messages = [AIMessage(content=f"Please tell how can I help", name="LLMModel")]
messages.append(HumanMessage(content=f"I want to learn coding", name="Cris"))
messages.append(AIMessage(content=f"Please tell which programming language you want to know", name="LLMModel"))
messages.append(HumanMessage(content=f"I want to learn C++ language", name="LLMModel"))

for message in messages:
    message.pretty_print()

================================== Ai Message ==================================
Name: LLMModel

Please tell how can I help
================================ Human Message =================================
Name: Cris

I want to learn coding
================================== Ai Message ==================================
Name: LLMModel

Please tell which programming language you want to know
================================ Human Message =================================
Name: LLMModel

I want to learn C++ language


In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="openai/gpt-oss-20b")

In [7]:
response = llm.invoke(messages)
response.content

'### 🚀 Quick‑Start Roadmap for Learning C++ (2025)\n\n| Stage | What you’ll learn | Why it matters | Suggested resources | Practice ideas |\n|-------|-------------------|----------------|---------------------|----------------|\n| **1️⃣ Setup & Basics** | Install a compiler, IDE, and write “Hello, World!” | You need a working environment and the syntax of C++ fundamentals. | • **IDE**: Visual Studio Code (VS\u202fCode) + C/C++ extension<br>• **Compiler**: GCC (Linux/macOS) or MSVC (Windows)<br>• **Tutorial**: “C++ Primer” – Chapters 1‑3 or the free **cppreference.com** “Getting Started” guide | Write a program that prints your name, age, and a short message. |\n| **2️⃣ Core Language Features** | Variables, data types, operators, control flow, functions, arrays, pointers, references | These are the building blocks of every C++ program. | • **Book**: *C++ Primer* (Chapters 4‑8)<br>• **Online**: **LearnCpp.com** (chapters 4‑12)<br>• **Video**: “C++ Programming Basics” (YouTube – freeCodeCa

In [8]:
print(response.content)

### 🚀 Quick‑Start Roadmap for Learning C++ (2025)

| Stage | What you’ll learn | Why it matters | Suggested resources | Practice ideas |
|-------|-------------------|----------------|---------------------|----------------|
| **1️⃣ Setup & Basics** | Install a compiler, IDE, and write “Hello, World!” | You need a working environment and the syntax of C++ fundamentals. | • **IDE**: Visual Studio Code (VS Code) + C/C++ extension<br>• **Compiler**: GCC (Linux/macOS) or MSVC (Windows)<br>• **Tutorial**: “C++ Primer” – Chapters 1‑3 or the free **cppreference.com** “Getting Started” guide | Write a program that prints your name, age, and a short message. |
| **2️⃣ Core Language Features** | Variables, data types, operators, control flow, functions, arrays, pointers, references | These are the building blocks of every C++ program. | • **Book**: *C++ Primer* (Chapters 4‑8)<br>• **Online**: **LearnCpp.com** (chapters 4‑12)<br>• **Video**: “C++ Programming Basics” (YouTube – freeCodeCamp) | Creat